In [102]:
# SET-UP - only need to run once

import numpy as np
import math

def unsigned_stirling_num_recc(n,k):        # computes the unsigned Stirling numbers of the first kind recursively, see wiki page below
    if k == 0 and n == 0:
        return 1
    elif k == 0 or n == 0:
        return 0
    else:
        return (n-1)*unsigned_stirling_num_recc(n-1,k) + unsigned_stirling_num_recc (n-1,k-1)


def basis_poly(i,d):      # returns the polynomial d!*(x choose i) as a vector of the form [a_0, a_1, ..., a_{d-1}, a_d]
    # Uses the expansion provided by the following wikipedia page: https://en.wikipedia.org/wiki/Stirling_numbers_of_the_first_kind
    p = np.zeros(d+1)
    for j in range(i+1):
        p[j] = ((-1)**(i-j)*unsigned_stirling_num_recc(i,j)/math.factorial(i))*math.factorial(d)
    return p

def GCD(v):     # recursively calculates the GCD of all the elements of v
    if len(v)==1:
        return int(v[0])
    else:
        return int(math.gcd(int(v[0]),GCD(v[1:])))


def print_poly(poly_v):         # prints the polynomial with coefficients given by poly_v into a "nice" format
    d = len(poly_v)-1
    
    # poly_v = poly_v*math.factorial(d)  # multiply by d! so that the coeff are integers
    gcd = GCD(poly_v)
    poly_v = poly_v/gcd         # simplifies the polynomial
    print("(",end="")
    for i in range(d+1):        # prints term-by-term, in our nice format
        if poly_v[i]:
            if i==0:
                print(int(poly_v[i]),end='') # prints out the constant term
            elif i==1:
                if poly_v[i] == 1:
                    print("+x",end='') # prints out the x term without the power and without the coeff, but with a plus sign if it is 1
                elif poly_v[i] == -1:
                    print("-x",end='') # prints out the x term without the power and without the coeff, but with a minus sign if it is -1
                else:
                    print(('{:+}'.format(int(poly_v[i])))+"x",end='') # prints out the x term without the power
            else:
                if poly_v[i] == 1:
                    print("+x^"+str(i),end='') # prints out the x^i term without the coeff, but with a plus sign if it is 1
                elif poly_v[i] == -1:
                    print("-x^"+str(i),end='') # prints out the x^i term without the coeff, but with a minus sign if it is -1
                else:
                    print(('{:+}'.format(int(poly_v[i])))+"x^"+str(i),end='') # prints out the x^i term
    print(")/"+str(int(math.factorial(d)/gcd)))          # finish it by dividing by d!, so that the expression looks nicer

def translate(v,min_val):          
    # gives the coefficients of p(x-1)+min_val+1 in terms of x, being given the coefficients of p and the minimum integer value it takes on {0,...,d+e}
    trans_v = np.zeros(len(v))
    d = len(v)-1
    for j in range(len(trans_v)):
        for i in range(j,len(trans_v)):
            trans_v[j]+=((-1)**(i-j))*v[i]*math.comb(i,j)       # the x^j term in a_i*(x-1)^j  (where p(x) = a_0 + ... + a_d*x^d)
    trans_v[0]+=(1-min_val)*math.factorial(d)        # makes sure the min value achieved is precisely 1 by subtractinf the initial min and adding 1
    return trans_v


def poly(v,vals):               # outputs the polynomial given by the vector v
                                # vals is the values taken by this poly on {0,...,d+e}, used in rescaling it
    d = len(v)-1
    poly_v = np.zeros(d+1)
    for i in range(len(v)):
        poly_v += v[i]*basis_poly(i,d)      # stores the polynomial coefficients in the form d!*[a_0, a_1, ..., a_{d-1}, a_d] 
                                            #       (the d! is to prevent rounding errors)

    print("Untranslated polynomial is: ",end="")
    print_poly(poly_v)

    min_val = min(vals)
    compress_poly = translate(poly_v,min_val)       # translates the polynomial above to get the actual dynamically compressing polynomial
    print("This translates to dynamically compressing polynomial: ",end="")
    print_poly(compress_poly)
    print("          --------              ")


In [96]:
# d = 2:
poly([3,-3,1],[3, 0, -2, -3, -3, -2, 0, 3])

Untranslated polynomial is: (6-7x+x^2)/2
This translates to dynamically compressing polynomial: (22-9x+x^2)/2
          --------              


In [98]:
# d = 3:
poly([-5,7,-4,1],[-5, 2, 5, 5, 3, 0, -3, -5, -5, -2, 5])

Untranslated polynomial is: (-30+56x-15x^2+x^3)/6
This translates to dynamically compressing polynomial: (-66+89x-18x^2+x^3)/6
          --------              


In [99]:
# d = 4:
poly([-1,-2,4,-3,1],[-1, -3, -1, 2, 4, 4, 2, -1, -3, -1])
poly([5,-6,5,-3,1],[5, -1, -2, -1, 0, 0, -1, -2, -1, 5])
poly([4,-8,9,-6,2],[4, -4, -3, 1, 4, 4, 1, -3, -4, 4])

Untranslated polynomial is: (-24-126x+95x^2-18x^3+x^4)/24
This translates to dynamically compressing polynomial: (312-374x+155x^2-22x^3+x^4)/24
          --------              
Untranslated polynomial is: (120-234x+107x^2-18x^3+x^4)/24
This translates to dynamically compressing polynomial: (552-506x+167x^2-22x^3+x^4)/24
          --------              
Untranslated polynomial is: (48-180x+101x^2-18x^3+x^4)/12
This translates to dynamically compressing polynomial: (408-440x+161x^2-22x^3+x^4)/12
          --------              


In [103]:
# d = 5:
poly([-4,8,-10,8,-4,1],[-4, 4, 2, -2, -4, -3, 0, 3, 4, 2, -2, -4, 4])

Untranslated polynomial is: (-480+2024x-1350x^2+315x^3-30x^4+x^5)/120
This translates to dynamically compressing polynomial: (-3600+5794x-2485x^2+445x^3-35x^4+x^5)/120
          --------              


In [104]:
# d = 6:
poly([2, -4, 8, -10, 8, -4, 1],[2, -2, 2, 4, 2, -2, -5, -5, -2, 2, 4, 2, -2, 2])

Untranslated polynomial is: (1440-10296x+10594x^2-3705x^3+565x^4-39x^5+x^6)/720
This translates to dynamically compressing polynomial: (30960-45060x+25504x^2-6375x^3+775x^4-45x^5+x^6)/720
          --------              


In [105]:
# d =7:
poly([-1, 2, -4, 8, -10, 8, -4, 1],[-1, 1, -1, 1, 5, 7, 5, 0, -5, -7, -5, -1, 1, -1, 1])

Untranslated polynomial is: (-5040+58344x-79576x^2+39004x^3-8575x^4+931x^5-49x^6+x^7)/5040
This translates to dynamically compressing polynomial: (-151200+373764x-258104x^2+83629x^3-14000x^4+1246x^5-56x^6+x^7)/5040
          --------              
